In [105]:
from ipywidgets import HTML
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, LogTicker, ColorBar, Legend, LegendItem, Label
import pandas as pd
import numpy as np

def calc_hp(hp_iv, hp_ev, hp_base, level):
    return int(((2*hp_base + hp_iv + int(hp_ev/4)) * level)/100) + level + 10

def calc_speed(x):
    temp = 2 * x['SPEED'] + x['SPEED_IV'] + int(x['SPEED_EV']/4)
    temp2 = int(temp * 50/100)
    temp3 = temp2 + 5
    return int(temp3 * x['SPEED_NATURE'])

df_base = pd.read_csv("convertcsv.csv")

df_base['SPEED_IV'] = 0
df_base['SPEED_EV'] = 0
df_base['SCARFED'] = False
df_base['TAILWIND'] = False
df_base['SPEED_NATURE'] = 1.0
df_base['SPEED_VAL'] = df_base.apply(calc_speed, axis=1)

def define_speed(iv, ev, nature, scarfed, tailwind):
    df_ = df_base.copy()
    df_['SCARFED'] = scarfed
    df_['TAILWIND'] = tailwind
    df_['SPEED_IV'] = iv
    df_['SPEED_EV'] = ev
    df_['SPEED_NATURE'] = nature
    df_['SPEED_VAL'] = df_.apply(calc_speed, axis=1)
    if scarfed:
        df_['SPEED_VAL'] *= 1.5
    if tailwind:
        df_['SPEED_VAL'] *= 2

    df_['SPEED_VAL'] = df_['SPEED_VAL'].astype(int)
    return df_

def define_specific(pokemon, iv, ev, nature, scarfed, tailwind):
    df_ = df_base.copy()
    df_ = df_[df_['Name'] == pokemon]
    df_['SCARFED'] = scarfed
    df_['TAILWIND'] = tailwind
    df_['SPEED_IV'] = iv
    df_['SPEED_EV'] = ev
    df_['SPEED_NATURE'] = nature
    df_['SPEED_VAL'] = df_.apply(calc_speed, axis=1)
    if scarfed:
        df_['SPEED_VAL'] *= 1.5
    if tailwind:
        df_['SPEED_VAL'] *= 2

    df_['SPEED_VAL'] = df_['SPEED_VAL'].astype(int)
    return df_

# append standard ev spreads + natures
df = df_base.copy()
df = df.append(define_speed(31, 252, 1.1, False, False))
df = df.append(define_speed(31, 252, 1.0, False, False))
df = df.append(define_speed(0, 0, 1.0, False, False))
df = df.append(define_speed(0, 0, 0.9, False, False))
df = df.append(define_speed(31, 252, 1.1, True, False))
df = df.append(define_speed(31, 252, 1.1, False, True))
df = df.append(define_speed(31, 252, 1.1, True, True))

# appending specific pokemon evs
# df = df.append(define_specific("Fuecoco", 22, 23, 1.0, False, False))


# define pokemon you want to lookup against
my_pkm = define_specific("Garchomp", 31, 252, 1.0, False, False)

# filter only for relevant pokemon in range of my_pkm speed value.
margin_low = 20 #define a speed margin for the lookup (lower and upper)
margin_high = 50

df = df[(df['SPEED_VAL'] >= my_pkm.iloc[0]['SPEED_VAL'] - margin_low)
        & (df['SPEED_VAL'] <= my_pkm.iloc[0]['SPEED_VAL'] + margin_high)]

from bokeh.plotting import figure, output_file, show

colormap = {True : {True : "indigo", False : "darkblue"},
            False : {True: "mediumorchid", False : "salmon"}}

colors = [colormap[x][y] for x,y in zip(df['SCARFED'], df['TAILWIND'])]
color_single = ['green']

source = ColumnDataSource(data=dict(
    y=df['SPEED_EV'],
    x=df['SPEED_VAL'],
    iv=df['SPEED_IV'],
    desc=df['Name'],
    scarfed=df['SCARFED'],
    tw=df['TAILWIND'],
    nature=df['SPEED_NATURE'],
    color=colors
))

single = ColumnDataSource(data=dict(
    y=my_pkm['SPEED_EV'],
    x=my_pkm['SPEED_VAL'],
    iv=my_pkm['SPEED_IV'],
    desc=my_pkm['Name'],
    scarfed=my_pkm['SCARFED'],
    tw=my_pkm['TAILWIND'],
    nature=my_pkm['SPEED_NATURE'],
    color=color_single
))

TOOLTIPS = [
    ("Speed:", "@x"),
    ("Name", "@desc"),
    ("Scarfed", "@scarfed"),
    ("Tailwind", "@tw"),
    ("IV", "@iv"),
    ("EV", "@y"),
    ("Nature", "@nature")
]
# create a new plot with the toolbar below
p = figure(width=1400, height=400,
           title=None, toolbar_location="below", tooltips=TOOLTIPS, align="center")

p.circle('x', 'y', size=10, source=source, color='color')
p.circle('x', 'y', size=10, source=single, color="color")

show(p)